# Motivação
Até o meio do século XX, todos os métodos de criptografia se baseavam num mesmo mecanismo. Dos mais simples, como a cifra de césar, até coisas como o blowfish, todos estes algoritmos, chamados de **Algoritmos de Chave Simétrica**, trabalham da mesma forma: o remetente realiza uma transformação de uma informação em função de uma chave **K** e transmitem o resultado. O receptor, em posse da mesma chave, realiza uma transformação inversa para conseguir acesso à informação. Esse método gera dois grandes problemas: 

1. Todos os participantes da "conversa" precisam ter a chave **K**, que precisa ser transmitida de forma segura entre eles, que não necessariamente é fácil, mesmo tendo acesso físico uns aos outros.

2. Considerando várias chaves para vários parceiros de conversa, a gerência de diferentes chaves também precisa presar pela segurança e distribuição de cada uma delas, algo extremamente complicado em grande escala.

Tendo em mente estes problemas, [James H. Ellis](https://en.wikipedia.org/wiki/James_H._Ellis) teorizou um sistema de criptografia que usasse duas chaves

In [15]:
frase = "Somebody once told me the world is gonna roll me"
for letra in frase:
    print(letra)

S
o
m
e
b
o
d
y
 
o
n
c
e
 
t
o
l
d
 
m
e
 
t
h
e
 
w
o
r
l
d
 
i
s
 
g
o
n
n
a
 
r
o
l
l
 
m
e


# Bibliografia
* [Curso de fundamentos de segurança de computadores - UTA - aula 44](https://www.cs.utexas.edu/users/byoung/cs361/lecture44.pdf)
* [Paper de James Ellis sobre Encriptação Não-secreta](https://web.archive.org/web/20030610193721/http://jya.com/ellisdoc.htm)
* Livro do collier
* Various wikipedia pages